In [12]:
import asyncio
import nest_asyncio
import numpy as np
import os
from playwright.async_api import async_playwright
import requests
from bs4 import BeautifulSoup
import pandas as pd
nest_asyncio.apply()

In [16]:
# Cargamos la información necesaria para el procesamiento
base_creditos = pd.read_excel('../01_Datos/solicitudes.xlsx')
Valor_simular = base_creditos.copy()

In [18]:
Valor_simular = Valor_simular.rename(columns=lambda x: x.replace(' ', ''))
Valor_simular

,ValorCredito,Plazo
0,2000000,12
1,2000000,24
2,2000000,36
3,2000000,48
4,3000000,24
5,3000000,36
6,4500000,6
7,4500000,24
8,10000000,24
9,10000000,36


In [4]:
import pandas as pd
import asyncio
from playwright.async_api import async_playwright

async def simular_creditos(df):
    # Lista para almacenar los resultados
    resultados = []
    
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)  # Visible para depuración
        page = await browser.new_page()
        await page.goto('https://www.bancodeoccidente.com.co/solicitarcredito/#/simuladorCreditoConsumo')
        
        for index, row in df.iterrows():
            valor_credito = str(row['ValorCredito'])
            plazo = str(row['Plazo'])
            print(f"Fila {index + 1}/{len(df)}: Simulando ValorCredito={valor_credito}, Plazo={plazo}")
            
            try:
                await asyncio.sleep(2)
                await page.wait_for_selector("a[ng-click=\"laf.irEstado('irSimuladorCreditoConsumo')\"]", timeout=10000)
                await asyncio.sleep(2)
                await page.click("a[ng-click=\"laf.irEstado('irSimuladorCreditoConsumo')\"]")
                await asyncio.sleep(2)
                await page.wait_for_selector("input[name='monto']", timeout=10000)
                await page.click("input[name='monto']", force=True)
                await page.fill("input[name='monto']", "")
                await page.fill("input[name='monto']", valor_credito)
                await page.wait_for_selector("select[name='plazo']", timeout=10000)
                await page.click("select[name='plazo']")
                await page.select_option("select[name='plazo']", value=plazo)
                await page.wait_for_selector("span#resultado b.ng-binding", timeout=15000)
                valor_cuota = await page.eval_on_selector("span#resultado b.ng-binding", "el => el.innerText")
                
                resultados.append({
                    'ValorCredito': valor_credito,
                    'Plazo': plazo,
                    'valor_cuota': valor_cuota
                })
                print(f"Resultado: {valor_cuota}")
                
            except Exception as e:
                print(f"Fila {index + 1}: No se pudo realizar la simulación ({str(e)})")
                resultados.append({
                    'ValorCredito': valor_credito,
                    'Plazo': plazo,
                    'valor_cuota': 'No se pudo realizar la simulación'
                })
            
            await page.goto('https://www.bancodeoccidente.com.co/solicitarcredito/#/')
            await asyncio.sleep(2)
        
        await browser.close()
    
    # Crear el DataFrame de resultados
    df_resultados = pd.DataFrame(resultados)
    return df_resultados

# Ejecutar la simulación
async def main():
    if 'Valor_simular' not in globals():
        raise ValueError("El DataFrame 'Valor_simular' no está definido. Cárgalo primero.")
    
    print("DataFrame original:")
    print(Valor_simular)
    
    # Ejecutar la simulación y asignar a una variable global
    global df_resultados_global
    df_resultados_global = await simular_creditos(Valor_simular)
    
    # Mostrar resultados
    print("\nResultados de la simulación:")
    print(df_resultados_global)
    
    # Guardar como CSV y Pickle para acceso desde otros notebooks
    #df_resultados_global.to_csv('resultados_simulacion.csv', index=False)
    #df_resultados_global.to_pickle('resultados_simulacion.pkl')
    #print("Resultados guardados en 'resultados_simulacion.csv' y 'resultados_simulacion.pkl'")

# Correr el script
if __name__ == "__main__":
    asyncio.run(main())

DataFrame original:
   ValorCredito  Plazo
0       2000000     12
1       2000000     24
2       2000000     36
3       2000000     48
4       3000000     24
5       3000000     36
6       4500000      6
7       4500000     24
8      10000000     24
9      10000000     36
Fila 1/10: Simulando ValorCredito=2000000, Plazo=12
Resultado: $196.038
Fila 2/10: Simulando ValorCredito=2000000, Plazo=24
Resultado: $112.941
Fila 3/10: Simulando ValorCredito=2000000, Plazo=36
Resultado: $86.091
Fila 4/10: Simulando ValorCredito=2000000, Plazo=48
Fila 4: No se pudo realizar la simulación (Page.select_option: Timeout 30000ms exceeded.
Call log:
  - waiting for locator("select[name='plazo']")
  -     - locator resolved to 2 elements. Proceeding with the first one: <select id="plazo" required="" name="plazo" ng-model="scc.simulacion.plazo" class="form-control input-sm ng-pristine ng-untouched ng-valid ng-not-empty ng-valid-required">…</select>
  -   - attempting select option action
  -     2 × waitin

In [5]:
df_resultados_global

,ValorCredito,Plazo,valor_cuota
0,2000000,12,$196.038
1,2000000,24,$112.941
2,2000000,36,$86.091
3,2000000,48,No se pudo realizar la simulación
4,3000000,24,$169.411
5,3000000,36,$129.136
6,4500000,6,No se pudo realizar la simulación
7,4500000,24,$254.116
8,10000000,24,$564.703
9,10000000,36,$430.453


In [6]:
df_resultados_global.copy()

,ValorCredito,Plazo,valor_cuota
0,2000000,12,$196.038
1,2000000,24,$112.941
2,2000000,36,$86.091
3,2000000,48,No se pudo realizar la simulación
4,3000000,24,$169.411
5,3000000,36,$129.136
6,4500000,6,No se pudo realizar la simulación
7,4500000,24,$254.116
8,10000000,24,$564.703
9,10000000,36,$430.453


**Funcion de limpieza resultados globales**

In [8]:
#funcion de limpieza de columna valor cuota


def limpiar_a_enteros(df_resultados_global, valor_cuota):
    """
    Limpia una columna string:
    - Elimina el signo '$'
    - Quita los valores decimales (deja solo la parte entera)
    - Convierte a enteros
    - Reemplaza valores NaN, vacíos o texto no numérico con 0
    """
    def convertir(valor_cuota):
        if pd.isna(valor_cuota) or valor_cuota == '' or not any(char.isdigit() for char in str(valor_cuota)):
            return 0  # Si es NaN, vacío o texto sin números, convertir a 0
        
        valor_cuota = str(valor_cuota).replace('$', '').replace('.', '')  # Eliminar '$' y ','
        valor_cuota = valor_cuota.split(',')[0]  # Tomar solo la parte entera antes del punto
        
        try:
            return int(valor_cuota)  # Convertir a entero
        except ValueError:
            return 0  # Si falla la conversión, devolver 0

    df_resultados_global[valor_cuota] = df_resultados_global[valor_cuota].apply(convertir)
    return df_resultados_global
  

#aplicar la funcion
df_resultados_global = limpiar_a_enteros(df_resultados_global, 'valor_cuota')

print(df_resultados_global)


  ValorCredito Plazo  valor_cuota
0      2000000    12       196038
1      2000000    24       112941
2      2000000    36        86091
3      2000000    48            0
4      3000000    24       169411
5      3000000    36       129136
6      4500000     6            0
7      4500000    24       254116
8     10000000    24       564703
9     10000000    36       430453


**Exportar resultados**

In [10]:
# Definir la ruta de la carpeta donde se guardará el archivo
ruta_carpeta = ("../03_Resultados" ) # ruta del archivo
nombre_archivo = "Resultados_tasa.xlsx"  # Nombre del archivo Excel
ruta_completa = os.path.join(ruta_carpeta, nombre_archivo)  

# Guardar el DataFrame en un archivo Excel
df_resultados_global.to_excel(ruta_completa, index=False, sheet_name="Resultados_tasa")

print(f"Archivo Excel guardado en: {ruta_completa}")

Archivo Excel guardado en: ../03_Resultados\Resultados_tasa.xlsx


Oportunidades de mejora
- Un try catch para validar la disponibilidad de la pagina para descarga
- Validar y almacenar los notebook para guardar como funciones y guardarlo con extensión py para extraer la data
- Realizar el análisis descriptivo de la data
- Validar cual va a ser el procedimiento y definir el paso a paso
- 

In [20]:
import pandas as pd
import asyncio
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout

async def simular_creditos(df):
    resultados = []  # Lista para almacenar los resultados
    url = 'https://www.bancodeoccidente.com.co/solicitarcredito/#/simuladorCreditoConsumo'
    intentos_maximos = 5
    
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)  # Visible para depuración
        page = None
        
        # Intentar conectar a la página hasta 5 veces
        for intento in range(intentos_maximos):
            try:
                page = await browser.new_page()
                print(f"Intento {intento + 1}/{intentos_maximos}: Conectando a {url}...")
                await page.goto(url, timeout=10000)
                print(f"Conexión exitosa en el intento {intento + 1}.")
                break  # Si la página carga, salir del bucle
            except PlaywrightTimeout:
                mensaje = f"Intento {intento + 1}/{intentos_maximos}: No se pudo conectar a la página. Reintentando..."
                print(mensaje)
                if intento == intentos_maximos - 1:  # Último intento
                    print("ERROR: No se pudo realizar la conexión después de 5 intentos.")
                    await browser.close()
                    resultados = [{'ValorCredito': str(row['ValorCredito']), 
                                  'Plazo': str(row['Plazo']), 
                                  'valor_cuota': 'No se pudo conectar al simulador'} 
                                  for index, row in df.iterrows()]
                    df_resultados = pd.DataFrame(resultados)
                    df_resultados.to_csv('resultados_simulacion.csv', index=False)
                    print("Archivo de resultados generado con error: resultados_simulacion.csv")
                    return df_resultados
                await asyncio.sleep(2)  # Esperar antes de reintentar
        
        # Si se conectó, proceder con la simulación
        if page:
            for index, row in df.iterrows():
                valor_credito = str(row['ValorCredito'])
                plazo = str(row['Plazo'])
                print(f"Fila {index + 1}/{len(df)}: Simulando ValorCredito={valor_credito}, Plazo={plazo}")
                
                try:
                    await asyncio.sleep(2)
                    await page.wait_for_selector("a[ng-click=\"laf.irEstado('irSimuladorCreditoConsumo')\"]", timeout=10000)
                    await asyncio.sleep(2)
                    await page.click("a[ng-click=\"laf.irEstado('irSimuladorCreditoConsumo')\"]")
                    await asyncio.sleep(2)
                    await page.wait_for_selector("input[name='monto']", timeout=10000)
                    await page.click("input[name='monto']", force=True)
                    await page.fill("input[name='monto']", "")
                    await page.fill("input[name='monto']", valor_credito)
                    await page.wait_for_selector("select[name='plazo']", timeout=10000)
                    await page.click("select[name='plazo']")
                    await page.select_option("select[name='plazo']", value=plazo)
                    await page.wait_for_selector("span#resultado b.ng-binding", timeout=15000)
                    valor_cuota = await page.eval_on_selector("span#resultado b.ng-binding", "el => el.innerText")
                    
                    resultados.append({
                        'ValorCredito': valor_credito,
                        'Plazo': plazo,
                        'valor_cuota': valor_cuota
                    })
                    print(f"Resultado: {valor_cuota}")
                
                except Exception as e:
                    print(f"Fila {index + 1}: No se pudo realizar la simulación ({str(e)})")
                    resultados.append({
                        'ValorCredito': valor_credito,
                        'Plazo': plazo,
                        'valor_cuota': 'No se pudo realizar la simulación'
                    })
                
                await page.goto('https://www.bancodeoccidente.com.co/solicitarcredito/#/')
                await asyncio.sleep(2)
            
            await browser.close()
        
        # Crear el DataFrame de resultados
        df_resultados = pd.DataFrame(resultados)
        return df_resultados

# Ejecutar la simulación
async def main():
    if 'Valor_simular' not in globals():
        raise ValueError("El DataFrame 'Valor_simular' no está definido. Cárgalo primero.")
    
    print("DataFrame original:")
    print(Valor_simular)
    
    # Ejecutar la simulación y asignar a una variable global
    global df_resultados_global
    df_resultados_global = await simular_creditos(Valor_simular)
    
    # Mostrar resultados
    print("\nResultados de la simulación:")
    print(df_resultados_global)
    
    # Guardar resultados
    df_resultados_global.to_csv('resultados_simulacion.csv', index=False)
    print("Resultados guardados en 'resultados_simulacion.csv'")

# Correr el script
if __name__ == "__main__":
    asyncio.run(main())

DataFrame original:
   ValorCredito  Plazo
0       2000000     12
1       2000000     24
2       2000000     36
3       2000000     48
4       3000000     24
5       3000000     36
6       4500000      6
7       4500000     24
8      10000000     24
9      10000000     36
Intento 1/5: Conectando a https://www.bancodeoccidente.com.co/solicitarcredito/#/simuladorCreditoConsumo...
Conexión exitosa en el intento 1.
Fila 1/10: Simulando ValorCredito=2000000, Plazo=12
Resultado: $196.038
Fila 2/10: Simulando ValorCredito=2000000, Plazo=24
Resultado: $112.941
Fila 3/10: Simulando ValorCredito=2000000, Plazo=36
Resultado: $86.091
Fila 4/10: Simulando ValorCredito=2000000, Plazo=48
Fila 4: No se pudo realizar la simulación (Page.select_option: Timeout 30000ms exceeded.
Call log:
  - waiting for locator("select[name='plazo']")
  -     - locator resolved to 2 elements. Proceeding with the first one: <select id="plazo" required="" name="plazo" ng-model="scc.simulacion.plazo" class="form-control in